In [7]:
import numpy as np
from typing import List

## Part 1

In [8]:
def parse_input(input_file: str) -> np.array:
    with open(input_file, "r") as file:
        char_array = None

        for line in file:
            line_chars = list(line.strip()) 
            line_array = np.array(line_chars)

            # first line: assignment
            if char_array is None:
                char_array = line_array
            # 2nd to nth line: append to existing 2d-array
            else:
                char_array = np.vstack((char_array, line_array))

    #print(char_array)
    return char_array

In [9]:
char_array = parse_input("example.txt")
print(char_array)

[['4' '6' '7' '.' '.' '1' '1' '4' '.' '.']
 ['.' '.' '.' '*' '.' '.' '.' '.' '.' '.']
 ['.' '.' '3' '5' '.' '.' '6' '3' '3' '.']
 ['.' '.' '.' '.' '.' '.' '#' '.' '.' '.']
 ['6' '1' '7' '*' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '+' '.' '5' '8' '.']
 ['.' '.' '5' '9' '2' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '7' '5' '5' '.']
 ['.' '.' '.' '$' '.' '*' '.' '.' '.' '.']
 ['.' '6' '6' '4' '.' '5' '9' '8' '.' '.']]


In [10]:
def get_array_neighbors(char_array: np.array, row_index: int, column_index: int):
    # build buffer around the array with '.' entries
    buffered_array = np.pad(char_array, pad_width=1, mode='constant', constant_values='.')
    adjusted_row_index = row_index + 1
    adjusted_column_index = column_index + 1
    # extract the three-by-three zone of neigbors around the entry
    three_by_three_zone = buffered_array[adjusted_row_index - 1:adjusted_row_index + 2, adjusted_column_index - 1:adjusted_column_index + 2]
    neighbors = three_by_three_zone.flatten().tolist()
    # remove the entry itself
    del neighbors[4]
    return neighbors

In [11]:
get_array_neighbors(char_array, row_index=0, column_index=2)

['.', '.', '.', '6', '.', '.', '.', '*']

In [12]:
def check_number(char_array: np.array, row_index: int, column_index: int):
    """Check if the number is valid, i.e. it has a symbol neighbor"""
    neighbors = []
    numbers = []
    while char_array[row_index][column_index].isdigit():
        numbers.append(char_array[row_index][column_index])
        all_neighbors = get_array_neighbors(char_array, row_index, column_index)
        symbol_neighbors = [n for n in all_neighbors if not n.isdigit() and n != '.']
        neighbors.extend(symbol_neighbors)
        column_index += 1
        if column_index >= len(char_array[0]):
            break
    number_valid = True if len(neighbors) > 0 else False
    return numbers, number_valid

In [13]:
def find_numbers(char_array: np.array):
    """Find the numbers in the array"""
    width = len(char_array[0])
    numbers = []
    for row_index, row in enumerate(char_array):
        column_index = 0
        while column_index < width:
            entry = row[column_index]
            if entry.isdigit():
                found_numbers, number_valid = check_number(char_array, row_index, column_index)
                if number_valid:
                    combined_found_number = int("".join(found_numbers))
                    numbers.append(combined_found_number)
                column_index += len(found_numbers)
            else:
                column_index += 1
    return numbers


In [14]:
def solve_part1(input_file: str) -> int:
    char_array = parse_input(input_file)
    numbers = find_numbers(char_array)
    #print("found valid numbers:", numbers)
    return sum(numbers)

In [15]:
solve_part1("example.txt")

4361

In [118]:
#solve_part1("input.txt")

## Part 2

In [99]:
def get_starting_numbers(one_dim_array: np.array, reverse=False):
    """Get the starting numbers in the array until there is a non-number character-entry"""
    index = 0
    numbers = []
    if reverse:
        one_dim_array = one_dim_array[::-1]
    while index < len(one_dim_array) and one_dim_array[index].isdigit():
        numbers.append(one_dim_array[index])
        index += 1 
    if reverse:
        numbers = numbers[::-1]
    return numbers

In [105]:
def get_number_array_neighbors(char_array, row_index, column_index, location):
    """Get the row neighbors of the given index entry and expand to left and/or right while there are still numbers"""
    if location == 'upper':
        if row_index == 0:
            return []
        number_neighbor_chars = []
        # left upper
        number_neighbor_chars += get_starting_numbers(char_array[row_index - 1, :column_index], reverse=True)
        # upper 
        middle_entry = ' ' if not char_array[row_index - 1, column_index].isdigit() else char_array[row_index - 1, column_index]
        number_neighbor_chars.append(middle_entry)
        # right upper
        number_neighbor_chars += get_starting_numbers(char_array[row_index - 1][column_index + 1:])
    if location == 'lower':
        if row_index == len(char_array) - 1:
            return []
        number_neighbor_chars = []
        # left lower
        number_neighbor_chars += get_starting_numbers(char_array[row_index + 1, :column_index], reverse=True)
        # lower 
        middle_entry = ' ' if not char_array[row_index + 1, column_index].isdigit() else char_array[row_index + 1, column_index]
        number_neighbor_chars.append(middle_entry)
        # right lower
        number_neighbor_chars += get_starting_numbers(char_array[row_index + 1][column_index + 1:])
    if location == 'left':
        if column_index == 0:
            return []
        number_neighbor_chars = get_starting_numbers(char_array[row_index, :column_index], reverse=True)
    if location == 'right':
        if column_index == len(char_array[0]) - 1:
            return []
        number_neighbor_chars = get_starting_numbers(char_array[row_index, column_index + 1:])
    #print(number_neighbor_chars)
    #print('joined and split', "".join(number_neighbor_chars).split(' '))
    number_neighbors = [int(number_entry) for number_entry in "".join(number_neighbor_chars).split(' ') if len(number_entry) > 0]
    return number_neighbors

In [106]:
def check_gear(char_array: np.array, row_index: int, column_index: int) -> List[int]:
    upper_numbers = get_number_array_neighbors(char_array, row_index, column_index, location='upper')
    left_numbers = get_number_array_neighbors(char_array, row_index, column_index, location='left')
    right_numbers = get_number_array_neighbors(char_array, row_index, column_index, location='right')
    lower_numbers = get_number_array_neighbors(char_array, row_index, column_index, location='lower')
    return upper_numbers + left_numbers + right_numbers + lower_numbers

In [113]:
def solve_part2(input_file: str) -> int:
    char_array = parse_input(input_file)
    gear_coords = np.where(char_array == '*')
    gear_ratios = []
    for coord in zip(*gear_coords):
        print("gear at:", coord)
        found_numbers = check_gear(char_array, coord[0], coord[1])
        print("found adjacent numbers:", found_numbers)
        if len(found_numbers) == 2:
            gear_ratios.append(found_numbers[0] * found_numbers[1])
    return sum(gear_ratios)

In [114]:
char_array

array([['4', '6', '7', '.', '.', '1', '1', '4', '.', '.'],
       ['.', '.', '.', '*', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '3', '5', '.', '.', '6', '3', '3', '.'],
       ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.'],
       ['6', '1', '7', '*', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '+', '.', '5', '8', '.'],
       ['.', '.', '5', '9', '2', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '7', '5', '5', '.'],
       ['.', '.', '.', '$', '.', '*', '.', '.', '.', '.'],
       ['.', '6', '6', '4', '.', '5', '9', '8', '.', '.']], dtype='<U1')

In [115]:
solve_part2("example.txt")

gear at: (1, 3)
found adjacent numbers: [467, 35]
gear at: (4, 3)
found adjacent numbers: [617]
gear at: (8, 5)
found adjacent numbers: [755, 598]


467835

In [117]:
#solve_part2("input.txt")